In [39]:
import numpy as np
from numpy import linalg as li
import math

In [40]:
#Сумма всех внедиагональных элементов
def sum_all(A):
    n = len(A)
    sum = 0
    for i in range (n):
        for j in range (n):
            if i != j:
                sum +=A[i][j]
    return sum

In [41]:
#Печать диагональных элементов
def dial_el(A):
    n = len(A)
    sum = 0
    ans = np.zeros(n)
    for i in range (n):
        for j in range (n):
            if i == j:
                ans[i] = (A[i][j])
    return ans

In [42]:
def matrix_rotate(A, k, l):
    x = 2 * A[k][l] / (A[k][k] - A[l][l])
    fi = 1 / 2 * math.atan(x)
    cos = math.cos(fi)
    sin = math.sin(fi)
    H = np.eye(len(A))
    H[k][k] = cos
    H[k][l] = -sin
    H[l][k] = sin
    H[l][l] = cos
    return H

Метод Якоби с обнулением максимального элемента

In [43]:
def jacobi_max_el(A_, e):
    A = A_.copy() 
    N = len(A)
    max_el = 0
    count = 1

    for i in range(N-1):
        for j in range(i+1,N):
            if abs(A[i,j]) >= max_el:
                max_el = abs(A[i,j])
                k = i
                l = j  
    vector = matrix_rotate(A, k, l)
    H = matrix_rotate(A, k, l)
    A = np.matmul(np.matmul(H.transpose(), A), H)
    max_el = 0
    for i in range(N-1):
            for j in range(i+1,N):
                if abs(A[i,j]) >= max_el:
                    max_el = abs(A[i,j])
                    k = i
                    l = j
    while max_el > e:
        count+=1
        H = matrix_rotate(A, k, l)
        A = np.matmul(np.matmul(H.transpose(), A), H)
        vector = vector.dot(H)
        max_el = 0
        for i in range(N-1):
            for j in range(i+1,N):
                if abs(A[i,j]) >= max_el:
                    max_el = abs(A[i,j])
                    k = i
                    l = j
    ans = dial_el(A)
    return count, ans, vector

Метод Якоби с циклическим обнулением элементов

In [47]:
def jacobi_cycle(A_, e):
    A = A_.copy() 
    N = len(A)
    count = 1
    k = 0
    l = 1
    summ = 0
    vector = matrix_rotate(A, k, l)
    
    H = matrix_rotate(A, k, l)
    A = np.matmul(np.matmul(H.transpose(), A), H)
    summ = sum_all(A)
    k = 0
    l = 2
    while summ > e:
        count+=1
        
        H = matrix_rotate(A, k, l)
        A = np.matmul(np.matmul(H.transpose(), A), H)
        vector = vector.dot(H)
        
        if l < N - 1 and l + 1 != k:
            l += 1
        elif l < N - 2 and l + 1 == k:
            l += 2
        elif l < N - 1 and l + 1 == k and l + 1 == N - 1 and k < N - 1:
            k += 1
            l = 0
        elif k < N - 1:
            k +=1
            l = 0
        else:
            k = 0
            l = 1    

        summ = sum_all(A)
    ans = dial_el(A)
    return count, ans, vector

Допустимый диапазон собственных чисел

In [48]:
def gershgorin_circle(A):
    i = 0
    left = 100000
    right = -100000
    for row in A: 
        centre = row[i] 
        radius = np.sum(np.absolute(row))-np.absolute(row[i])
        if centre + radius > right:
            right = centre + radius
        if centre - radius < left:
            left = centre - radius
        return left, right

In [56]:
A = np.array([[1, 1/2, 1/3, 1/4, 1/5],
              [1/2, 1/3, 1/4, 1/5, 1/6],
              [1/3, 1/4, 1/5, 1/6, 1/7],
              [1/4, 1/5, 1/6, 1/7, 1/8],
              [1/5, 1/6, 1/7, 1/8, 1/9]])
e = 1.0e-5
l, r = gershgorin_circle(A)
print("Допустимый диапозон собственых значений: [",l,",", r,"]\n")
count1, w1, v1 = jacobi_max_el(A, e)
print("Количество итераций: ", count1)
print("Собственные числа: ", w1)
print("Собственные векторы:", v1)
count, w, v = jacobi_cycle(A, e)
print()
print("Количество итераций: ", count)
print("Собственные числа: ", w)
print("Собственные векторы:", v)


print()
print()
e = 1.0e-10
l, r = gershgorin_circle(A)
print("Допустимый диапозон собственых значений: [",l,",", r,"]\n")
count1, w1, v1 = jacobi_max_el(A, e)
print("Количество итераций: ", count1)
print("Собственные числа: ", w1)
print("Собственные векторы:", v1)
count, w, v = jacobi_cycle(A, e)
print()
print("Количество итераций: ", count)
print("Собственные числа: ", w)
print("Собственные векторы:", v)

Допустимый диапозон собственых значений: [ -0.2833333333333332 , 2.283333333333333 ]

Количество итераций:  24
Собственные числа:  [1.56705069e+00 2.08534219e-01 3.05898013e-04 3.28795815e-06
 1.14074916e-02]
Собственные векторы: [[ 0.76785473 -0.60187171  0.04716094 -0.00618627  0.21421283]
 [ 0.44579108  0.27591402 -0.43263259  0.11682232 -0.72410176]
 [ 0.3215783   0.42487637  0.66719715 -0.50636605 -0.12045269]
 [ 0.25343894  0.44390532  0.23325499  0.76711959  0.30957458]
 [ 0.20982262  0.42901053 -0.55771405 -0.37607876  0.56519398]]

Количество итераций:  21
Собственные числа:  [1.56703002e+00 2.08524842e-01 1.12361926e-05 1.14269989e-02
 3.08491833e-04]
Собственные векторы: [[ 0.76863679 -0.60184064  0.00653631  0.21099992 -0.04921014]
 [ 0.44303679  0.27436135 -0.10475935 -0.7265385   0.43544408]
 [ 0.32146099  0.43015099  0.49441283 -0.11649131 -0.6735122 ]
 [ 0.2543201   0.44006594 -0.77620909  0.30161316 -0.21952722]
 [ 0.21189622  0.4287491   0.37687949  0.56840138  0.5533

In [57]:
A = np.array([[1, 0, 7],
              [0, 2, 5],
              [7, 5, 3]])
e = 1.0e-5
l, r = gershgorin_circle(A)
print("Допустимый диапозон собственых значений: [",l,",", r,"]\n")
count1, w1, v1 = jacobi_max_el(A, e)
print("Количество итераций: ", count1)
print("Собственные числа: ", w1)
print("Собственные векторы:", v1)
count, w, v = jacobi_cycle(A, e)
print()
print("Количество итераций: ", count)
print("Собственные числа: ", w)
print("Собственные векторы:", v)


print()
print()
e = 1.0e-10
l, r = gershgorin_circle(A)
print("Допустимый диапозон собственых значений: [",l,",", r,"]\n")
count1, w1, v1 = jacobi_max_el(A, e)
print("Количество итераций: ", count1)
print("Собственные числа: ", w1)
print("Собственные векторы:", v1)
count, w, v = jacobi_cycle(A, e)
print()
print("Количество итераций: ", count)
print("Собственные числа: ", w)
print("Собственные векторы:", v)

Допустимый диапозон собственых значений: [ -6 , 8 ]

Количество итераций:  7
Собственные числа:  [-6.48851243  1.66617063 10.82234179]
Собственные векторы: [[ 0.62726819 -0.57346466  0.52694677]
 [ 0.39526576  0.8174105   0.41905257]
 [-0.67104366 -0.05457433  0.73940655]]

Количество итераций:  5
Собственные числа:  [-6.44600931  1.62386541 10.8221439 ]
Собственные векторы: [[ 0.66711565 -0.52912404  0.52438961]
 [ 0.33528792  0.84185848  0.42291407]
 [-0.66523584 -0.1063111   0.73902587]]


Допустимый диапозон собственых значений: [ -6 , 8 ]

Количество итераций:  9
Собственные числа:  [-6.48851243  1.66617063 10.82234179]
Собственные векторы: [[ 0.62726819 -0.57346521  0.52694618]
 [ 0.39526576  0.81741006  0.41905342]
 [-0.67104366 -0.0545751   0.73940649]]

Количество итераций:  5
Собственные числа:  [-6.44600931  1.62386541 10.8221439 ]
Собственные векторы: [[ 0.66711565 -0.52912404  0.52438961]
 [ 0.33528792  0.84185848  0.42291407]
 [-0.66523584 -0.1063111   0.73902587]]


In [58]:
A = np.array([[1, 1/2, 1/3, 1/4, 1/5, 1/6, 1/7],
              [1/2, 1/3, 1/4, 1/5, 1/6, 1/7, 1/8],
              [1/3, 1/4, 1/5, 1/6, 1/7, 1/8, 1/9],
              [1/4, 1/5, 1/6, 1/7, 1/8, 1/9, 1/10],
              [1/5, 1/6, 1/7, 1/8, 1/9, 1/10, 1/11],
              [1/6, 1/7, 1/8, 1/9, 1/10, 1/11, 1/12],
              [1/7, 1/8, 1/9, 1/10, 1/11, 1/12, 1/13]])
e = 1.0e-5
l, r = gershgorin_circle(A)
print("Допустимый диапозон собственых значений: [",l,",", r,"]\n")
count1, w1, v1 = jacobi_max_el(A, e)
print("Количество итераций: ", count1)
print("Собственные числа: ", w1)
print("Собственные векторы:", v1)
count, w, v = jacobi_cycle(A, e)
print()
print("Количество итераций: ", count)
print("Собственные числа: ", w)
print("Собственные векторы:", v)


print()
print()
e = 1.0e-10
l, r = gershgorin_circle(A)
print("Допустимый диапозон собственых значений: [",l,",", r,"]\n")
count1, w1, v1 = jacobi_max_el(A, e)
print("Количество итераций: ", count1)
print("Собственные числа: ", w1)
print("Собственные векторы:", v1)
count, w, v = jacobi_cycle(A, e)
print()
print("Количество итераций: ", count)
print("Собственные числа: ", w)
print("Собственные векторы:", v)

Допустимый диапозон собственых значений: [ -0.5928571428571425 , 2.5928571428571425 ]

Количество итераций:  43
Собственные числа:  [1.66088534e+00 2.71920198e-01 1.00855961e-03 2.26279644e-07
 2.93975172e-05 2.79783121e-07 2.12897549e-02]
Собственные векторы: [[ 0.73322536 -0.62323822  0.07525525 -0.00152695 -0.0157357   0.00146848
   0.26084332]
 [ 0.43635978  0.16307078 -0.52781022  0.045809    0.22630946 -0.03810453
  -0.67055801]
 [ 0.31977939  0.32151004  0.42925521 -0.29875517 -0.62875506  0.19572037
  -0.29532754]
 [ 0.25488553  0.35737195  0.45823321  0.70646109  0.20533781 -0.23570973
   0.02304397]
 [ 0.21284385  0.35707697  0.17047292 -0.62751408  0.49479081 -0.32393004
   0.23368778]
 [ 0.18314277  0.34456586 -0.18138399  0.07848429  0.18432294  0.79960353
   0.36787553]
 [ 0.16093925  0.3281296  -0.50946457  0.09809985 -0.4819201  -0.40047955
   0.45235047]]

Количество итераций:  43
Собственные числа:  [1.66074074e+00 2.71836283e-01 4.61007004e-05 2.14341913e-02
 2.87950